## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Clyde River,CA,70.4692,-68.5914,-17.19,96,26,4.09,scattered clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,32.76,96,100,24.14,snow
2,2,Havelock,US,34.8791,-76.9013,68.99,92,95,10.00,overcast clouds
3,3,Butaritari,KI,3.0707,172.7902,82.56,72,100,17.83,overcast clouds
4,4,Kodiak,US,57.7900,-152.4072,9.91,56,1,42.57,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) 

What is the minimum temperature you would like for your trip? 74
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Butaritari,KI,3.0707,172.7902,82.56,72,100,17.83,overcast clouds
5,5,Hithadhoo,MV,-0.6000,73.0833,80.26,79,93,7.47,overcast clouds
16,16,Kapaa,US,22.0752,-159.3190,78.78,75,75,1.99,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()


City_ID        167
City           167
Country        166
Lat            167
Lng            167
Max Temp       167
Humidity       167
Cloudiness     167
Wind Speed     167
Description    167
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.copy()

#dropping any null values
clean_df.dropna(inplace=True)

# checking new data frame
clean_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Butaritari,KI,3.0707,172.7902,82.56,72,100,17.83,overcast clouds
5,5,Hithadhoo,MV,-0.6000,73.0833,80.26,79,93,7.47,overcast clouds
16,16,Kapaa,US,22.0752,-159.3190,78.78,75,75,1.99,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Butaritari,KI,82.56,overcast clouds,3.0707,172.7902,
5,Hithadhoo,MV,80.26,overcast clouds,-0.6000,73.0833,
16,Kapaa,US,78.78,broken clouds,22.0752,-159.3190,
18,Rikitea,PF,76.12,light rain,-23.1203,-134.9692,
20,Santa Rita,BR,79.47,few clouds,-7.1139,-34.9781,
23,Carnarvon,AU,80.67,clear sky,-24.8667,113.6333,
25,Kavieng,PG,84.00,light rain,-2.5744,150.7967,
27,Atuona,PF,79.25,overcast clouds,-9.8000,-139.0333,
31,Pangai,TO,80.80,clear sky,-19.8000,-174.3500,
32,Hilo,US,74.93,light rain,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df.copy()

clean_hotel_df.dropna(inplace=True)

clean_hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))